# Timescaledb example

In [ ]:
import psycopg2 as psy
from datetime import datetime, timedelta

In [ ]:
# create testdb
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

con = psy.connect("dbname=postgres host=timescaledb user=postgres password=foobar")

with con:
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    with con.cursor() as cur:
        cur.execute("DROP DATABASE IF EXISTS testdb;")
        cur.execute('CREATE DATABASE testdb')

# not sure if the AUTOCOMMIT is needed
with con:
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    with con.cursor() as cur:   
        cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb CASCADE;")
con.close()

In [ ]:
con = psy.connect("dbname=testdb host=timescaledb user=postgres password=foobar")

with con:
    with con.cursor() as cur:
        cur.execute("""
        CREATE TABLE conditions (
          time        TIMESTAMPTZ       NOT NULL,
          location    TEXT              NOT NULL,
          temperature DOUBLE PRECISION  NULL,
          humidity    DOUBLE PRECISION  NULL
        );""")


In [ ]:
with con:
    with con.cursor() as cur:
        cur.execute("SELECT create_hypertable('conditions', 'time');")

In [ ]:
SQL = "INSERT INTO conditions(time, location, temperature, humidity) VALUES (%s, %s, %s, %s)"
tstart = datetime.now()
tnow = tstart
delta = timedelta(seconds=60)
with con:
    for i in range(1000):
        with con.cursor() as cur:
            cur.execute(SQL, (tnow, "here", i%20, i%35))
        tnow = tnow + delta

In [ ]:
with con:
    with con.cursor() as cur:
        cur.execute("SELECT * FROM conditions ORDER BY time DESC LIMIT 10;")
        for x in cur.fetchall():
            print(x)

In [ ]:
SQL= """SELECT time_bucket('5 minute', time) AS five_min, count(*)
         FROM conditions WHERE time > '%s' GROUP BY five_min ORDER BY five_min LIMIT 5;"""

with con:
    with con.cursor() as cur:
        # cur.execute(SQL, (tstart))
        cur.execute(SQL % tstart)
        for x in cur.fetchall():
            print(x)